In this EDA I will perform hierarchical clustering of the scans in the sample subset based on the frequency of pixels with given radiodensitites (binning Hounsfield Units [HU]). I will display a heatmap showing the different frequencies of the HU bins. Finally, we will look to see if the clusters formed distinguish between scans with and without cancer. I do not expect this simple clustering to be able to discriminate between cancerous and non-cancerous scans, especially when performed on this small subset of the data, but it is an exercise to become comfortable with the data type, the Hounsfield Unit scale, and the notion that there will be an association between pixel values and diagnosis. Ultimately, I plan to rely on convolutional neural networks to learn diagnosis from the images.

I will use some of the same pre-processing code which I described in the EDA 1 notebook to read in the DICOM files and convert the pixels to HU.

In [14]:
%matplotlib inline

import numpy as np 
import pandas as pd
import dicom
import os
import scipy.ndimage
import matplotlib.pyplot as plt
import seaborn as sns;

from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# Some constants 
INPUT_FOLDER = '../input/sample_images/'
patients = os.listdir(INPUT_FOLDER)
patients.sort()

In [2]:
# This function will load the scan for a given patient and infer the slice thickness for each scan
def load_scan(path):
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

In [3]:
# This function will convert pixels to Hounsfield Units
def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices]) #make 3D arrays combining all slices for each patient 
    image = image.astype(np.int16) #convert to int16

    # Pixels that were outside the scanning bounds have been set to -2000
    # Reset to have HU of air = 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept #RescaleIntercept included in metadata of pic
        slope = slices[slice_number].RescaleSlope #RescaleIntercept included in metadata of pic
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

Read in each of 20 images and change to HU scale.

In [7]:
allscans = []
allhu = []
for patient in patients:
    patient_scan = load_scan(INPUT_FOLDER + patient)
    allscans.append(patient_scan)
    patient_hu = get_pixels_hu(patient_scan)
    allhu.append(patient_hu)

Flatten each patients scan to a single vector.

In [10]:
flathu = []
for hu in allhu:
    flathu.append(hu.flatten())

In [12]:
print(len(flathu))
print(flathu[0].shape) #each image is now a vector of length 35127296

20
(35127296,)


In [ ]:
sns.clustermap(flathu)